- 참고 링크 : https://gwkim.tistory.com/m/2

In [2]:
# 라이브러리 import 
from bs4 import BeautifulSoup
import requests
import time 
import random 
from tqdm import tqdm 
from collections import defaultdict 
import json
import pandas as pd

# 파라미터 및 관련 변수 설정
# 원하는 주소의 위 , 경도 주소가 필요!

#url = 'https://www.yogiyo.co.kr/api/v1/restaurants/1115073/menu/'
url = 'https://www.yogiyo.co.kr/api/v1/restaurants-geo/'
api_key = 'fe5183cc3dea12bd0ce299cf110a75a2'

## 특정 위경도의 "치킨" 카테고리를 조회 
parameters = {"category" : "한식" ,
#              "items" : 20 ,
            "lat" : 37.5565050755347,
            "lng" : 126.939656244325,
#             "order" : 'review_count', ## 정렬 기준
            "page" : 0
             }
headers = {'X-ApiSecret':api_key,'X-ApiKey':"iphoneap"}
res = requests.get(url,params=parameters, headers=headers)
info = json.loads(res.content)

In [24]:
with open('../프로젝트_최종/요기요.json', 'w') as outfile:
    json.dump(info, outfile, indent=4)

In [3]:
##  필요한 항목 
key_list =["id" , "name" , "address" , "estimated_delivery_time" ,"min_order_amount"]
## dataframe 만들기 
df_list =[]
for idx in range(len(info["restaurants"])):
    info_list=[]
    for key in key_list:
        info_list.append(info["restaurants"][idx][key])
    df_list.append(info_list)
    
## dataframe으로 표출하기 
df = pd.DataFrame(df_list , columns = key_list)

print(df.shape)  ##(280, 5)
print(df.info()) 

(100, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   id                       100 non-null    int64 
 1   name                     100 non-null    object
 2   address                  100 non-null    object
 3   estimated_delivery_time  100 non-null    object
 4   min_order_amount         100 non-null    int64 
dtypes: int64(2), object(3)
memory usage: 4.0+ KB
None


In [4]:
df.head(2)

,id,name,address,estimated_delivery_time,min_order_amount
0,1138033,엽전식당-연남점,서울특별시 마포구 연남동 227-34 1층,65~75분,9000
1,377422,백순대본가새맛-연대점,서울특별시 서대문구 창천동 52-149 1층,45~55분,5000
